In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

2024-12-29 12:43:18.733267: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-29 12:43:18.733764: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-29 12:43:18.736171: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-29 12:43:18.742027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735497798.751650    9536 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735497798.75

In [2]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
## Split the data into training,validation and test sets

train, test = train_test_split(data,test_size=0.20,random_state=42)


In [4]:
# train dataset
X_train = train.drop(['quality'],axis=1).values
y_train = train[['quality']].values.ravel()

## test dataset
X_test = test.drop(['quality'],axis=1).values
y_test = test[['quality']].values.ravel()

## splitting this train data into train and validation

X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train,test_size=0.20,random_state=42)

signature=infer_signature(X_train,y_train)

In [13]:
### ANN Model

def train_model(params,epochs,X_train,y_train,X_valid,y_valid,X_test,y_test):

    ## Define model architecture
    mean=np.mean(X_train,axis=0)
    var=np.var(X_train,axis=0)

    model=keras.Sequential(
        [
            keras.Input([X_train.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(X_train,y_train,validation_data=(X_valid,y_valid),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(X_valid,y_valid,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [14]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        X_train=X_train,
        y_train=y_train,
        X_valid=X_valid,
        y_valid=y_valid,
        X_test=X_test,
        y_test=y_test,
    )
    return result

In [15]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [16]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2024-12-29 13:40:11.671967: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


 1/49 ━━━━━━━━━━━━━━━━━━━━ 12s 260ms/step - loss: 43.5125 - root_mean_squared_error: 6.5964
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 40.7559 - root_mean_squared_error: 6.3836 - val_loss: 38.4296 - val_root_mean_squared_error: 6.1992

Epoch 2/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 37.9787 - root_mean_squared_error: 6.1627
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 36.8621 - root_mean_squared_error: 6.0713 - val_loss: 35.0294 - val_root_mean_squared_error: 5.9186

Epoch 3/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 34.4323 - root_mean_squared_error: 5.8679
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 33.4668 - root_mean_squared_error: 5.7850 - val_loss: 31.9575 - val_root_mean_squared_error: 5.6531

 1/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 31.5822 - root_mean_squared_error: 5.6198
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 31.9695 - root_mean_squared_error: 5.6

In [19]:
## Inferencing

from mlflow.models import validate_serving_input

model_uri = 'runs:/2727c8287641451c94926410b959f70e/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(X_test)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

/home/victor/anaconda3/envs/mlcourse/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[ 3.6471648],
       [ 7.7468877],
       [ 5.229952 ],
       [ 3.6116319],
       [ 5.1000714],
       [ 6.615336 ],
       [ 4.61668  ],
       [ 4.409793 ],
       [ 6.64171  ],
       [ 7.296145 ],
       [ 6.292575 ],
       [ 4.010434 ],
       [ 6.681763 ],
       [ 3.4976254],
       [ 6.133785 ],
       [ 3.5311275],
       [ 5.923035 ],
       [ 5.886807 ],
       [ 5.104904 ],
       [ 3.9514956],
       [ 4.090308 ],
       [ 5.8943768],
       [ 3.257913 ],
       [ 4.7266564],
       [ 3.8203654],
       [ 2.9490654],
       [ 3.6001482],
       [ 5.133659 ],
       [ 5.386474 ],
       [ 3.4834418],
       [ 3.7770672],
       [ 4.4607253],
       [ 4.700049 ],
       [ 3.4754214],
       [ 4.001106 ],
       [ 6.18245  ],
       [ 6.7971473],
       [ 4.6096115],
       [ 3.7089453],
       [ 5.460692 ],
       [ 4.1456995],
       [ 4.659162 ],
       [ 4.5097885],
       [ 6.5312047],
       [ 3.6531138],
       [ 4.2041864],
       [ 8.854989 ],
       [ 3.97

In [21]:
# Load model as a PyFuncModel.
model_uri = 'runs:/2727c8287641451c94926410b959f70e/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[ 3.6471648],
       [ 7.7468877],
       [ 5.229952 ],
       [ 3.6116319],
       [ 5.1000714],
       [ 6.615336 ],
       [ 4.61668  ],
       [ 4.409793 ],
       [ 6.64171  ],
       [ 7.296145 ],
       [ 6.292575 ],
       [ 4.010434 ],
       [ 6.681763 ],
       [ 3.4976254],
       [ 6.133785 ],
       [ 3.5311275],
       [ 5.923035 ],
       [ 5.886807 ],
       [ 5.104904 ],
       [ 3.9514956],
       [ 4.090308 ],
       [ 5.8943768],
       [ 3.257913 ],
       [ 4.7266564],
       [ 3.8203654],
       [ 2.9490654],
       [ 3.6001482],
       [ 5.133659 ],
       [ 5.386474 ],
       [ 3.4834418],
       [ 3.7770672],
       [ 4.4607253],
       [ 4.700049 ],
       [ 3.4754214],
       [ 4.001106 ],
       [ 6.18245  ],
       [ 6.7971473],
       [ 4.6096115],
       [ 3.7089453],
       [ 5.460692 ],
       [ 4.1456995],
       [ 4.659162 ],
       [ 4.5097885],
       [ 6.5312047],
       [ 3.6531138],
       [ 4.2041864],
       [ 8.854989 ],
       [ 3.97

In [22]:
## Register in the model registry
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1735501997633, current_stage='None', description=None, last_updated_timestamp=1735501997633, name='wine-quality', run_id='2727c8287641451c94926410b959f70e', run_link=None, source='file:///home/victor/Documents/ML_course/mlflowstarter/projects/simple_deep_learning_project/mlruns/758312869568036635/2727c8287641451c94926410b959f70e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>